<a href="https://colab.research.google.com/github/junggu429/house-cost-project/blob/main/%EC%B6%95%EA%B5%AC%202%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

원핫인코딩함

점수 1,212,798

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv('/content/drive/MyDrive/FIFA_train.csv')
test = pd.read_csv('/content/drive/MyDrive/FIFA_test.csv')
submission = pd.read_csv('/content/drive/MyDrive/submission.csv')

In [27]:
train.head()

,id,name,age,continent,contract_until,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves,value
0,0,L. Messi,31,south america,2021,ST,left,5.0,94,94,4.0,110500000.0
1,3,De Gea,27,europe,2020,GK,right,4.0,91,93,1.0,72000000.0
2,7,L. Suárez,31,south america,2021,ST,right,5.0,91,91,3.0,80000000.0
3,8,Sergio Ramos,32,europe,2020,DF,right,4.0,91,91,3.0,51000000.0
4,9,J. Oblak,25,europe,2021,GK,right,3.0,90,93,1.0,68000000.0


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8932 entries, 0 to 8931
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                8932 non-null   int64  
 1   name              8932 non-null   object 
 2   age               8932 non-null   int64  
 3   continent         8932 non-null   object 
 4   contract_until    8932 non-null   object 
 5   position          8932 non-null   object 
 6   prefer_foot       8932 non-null   object 
 7   reputation        8932 non-null   float64
 8   stat_overall      8932 non-null   int64  
 9   stat_potential    8932 non-null   int64  
 10  stat_skill_moves  8932 non-null   float64
 11  value             8932 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 837.5+ KB


In [9]:
train['continent'].value_counts()

europe           5322
south america    1927
asia              787
africa            721
oceania           175
Name: continent, dtype: int64

In [10]:
train['position'].value_counts()

MF    3428
DF    2791
ST    1705
GK    1008
Name: position, dtype: int64

In [29]:
train['contract_until'].value_counts()

2019            2366
2021            2308
2020            2041
2022             761
2023             506
Jun 30, 2019     501
2018             327
Dec 31, 2018      64
May 31, 2019      19
2024              12
Jan 31, 2019      10
Jun 30, 2020       9
2025               3
Jan 1, 2019        2
2026               1
Jan 12, 2019       1
May 31, 2020       1
Name: contract_until, dtype: int64

In [30]:
# 계약기간 전처리

def con_period(x):
    if x == 'Dec 31, 2018' :
        return '2019'
    elif x == 'Jun 30, 2020' :
        return '2020.5'
    elif x == 'Jun 30, 2019' :
        return '2019.5'
    elif x == 'May 31, 2020' :
        return '2020.3333'
    elif x == 'May 31, 2019' :
        return '2019.3333'
    elif x == 'Jan 31, 2019' :
        return '2019.0833'
    elif x == 'Jan 1, 2019' :
        return '2019'
    elif x == 'Jan 12, 2019' :
        return '2019.034'
    elif x == 'Dec 31, 2019' :
        return '2020'
    elif x == 'Jun 1, 2019' :
        return '2019.416'
    else :
        return x

In [32]:
train.contract_until = train.contract_until.apply(con_period).astype('float64') - 2018
test.contract_until = test.contract_until.apply(con_period).astype('float64') - 2018

In [ ]:
# 원핫 인코딩

In [34]:
train = pd.get_dummies(data = train, columns = ['continent'], prefix = 'continent')

In [36]:
train = pd.get_dummies(data = train, columns = ['position'], prefix = 'position')


In [37]:
train = pd.get_dummies(data = train, columns = ['prefer_foot'], prefix = 'prefer_foot')

In [39]:
test = pd.get_dummies(data = test, columns = ['continent'], prefix = 'continent')
test = pd.get_dummies(data = test, columns = ['position'], prefix = 'position')
test = pd.get_dummies(data = test, columns = ['prefer_foot'], prefix = 'prefer_foot')

In [42]:
X_train = train.drop(['id','name','value'], axis = 1)

In [44]:
y_train = train['value']

In [47]:
X_test = test.drop(['id','name'], axis = 1)

In [48]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [49]:
rf = RandomForestRegressor()
rf.fit(X_train_scaled,y_train)
rf_pred = rf.predict(X_test_scaled)

In [50]:
submission['value'] = rf_pred

In [51]:
submission.to_csv('soccer2.csv', index = False)